In [47]:
import os
import numpy as np
import pandas as pd
import pyesg
import streamlit as st
from functions import invest_series, salary_series, age_series, mortality_sim
from asset_mix import asset_mix

class Retirement_Analysis:
    def __init__(self, age, ret_age, salary, fund_value, ret_inc):
        self.age = age
        self.ret_age = ret_age
        self.salary = salary
        self.fund_value = fund_value
        self.ret_inc = ret_inc
        self.fund_projection()

    def fund_projection(self):
        # projection inputs
        yrs = 65                    # number of yrs to project
        freq = 1                    # number of periods per year
        n_scen = 100                # number of scenarios

        # demographic inputs
        age = self.age               # starting age
        sex = 'M'                    # sex
        smoker = 'NS'                # SM for smoker NS for non-smoker
        ret_age = self.ret_age       # age at which time investments into retirement fund cease and withdrawals begin
        mi = .99                     # mortality improvement

        # fund inputs
        fund_value = self.fund_value # starting value of retirement fund
        ret_inc = self.ret_inc       # desired annual retirement income
        salary = self.salary         # starting salary
        salary_growth = .045         # annual salary growth rate
        invest_pct = .15             # pct of salary to be invested in retirement
        inflation = .02              # annual inflation assumption
        s_pct = .80                  # starting pct of funds to be invested in stocks
        b_pct = 1 - s_pct            # starting pct of funds to be invested in bonds
        pct_5 = .55                  # pct of funds to be invested in stocks 5 yrs out from retirement
        s_pct_end = .1               # pct of funds to be invested in stocks at and during retirement
        b_pct_end = 1 - s_pct_end    # pct of funds to be invested in bonds at and during retirement

        # model inputs
        x0 = 100.0                  # the start value of our process
        dt = 1/freq                 # the length of each timestep in yrs
        n_steps = freq * yrs        # the number of time steps per scenario
        # random_state = 259        # optional random_state for reproducibility

        config = {
            'yrs':yrs,
            'freq':freq,
            'n_scen':n_scen,
            'age':age,
            'sex':sex,
            'smoker':smoker,
            'ret_age':ret_age,
            'mi':mi,   
            'fund_value':fund_value,
            'ret_inc':ret_inc,
            'salary':salary,
            'salary_growth':salary_growth,
            'invest_pct':invest_pct,
            'inflation':inflation,
            's_pct':s_pct,
            'b_pct':b_pct,
            'pct_5':pct_5,
            's_pct_end':s_pct_end,
            'b_pct_end':b_pct_end,
            'x0':x0,
            'dt':dt,
            'n_steps':n_steps
        }

        # instantiate a new model with the required parameters
        stock_model = pyesg.GeometricBrownianMotion(mu=0.10, sigma=0.15)
        bond_model = pyesg.GeometricBrownianMotion(mu=0.05, sigma=0.05)

        # run model for both equities and bonds
        s_model_results = stock_model.scenarios(x0, dt, n_scen, n_steps)
        b_model_results = bond_model.scenarios(x0, dt, n_scen, n_steps)

        # create stock and bond index return arrays. 
        stock_return = s_model_results[:, 1:] / s_model_results[:, :-1]
        bond_return = b_model_results[:, 1:] / b_model_results[:, :-1]

        # set beginning of fund array to starting investment value
        stock_array = np.insert(stock_return, 0, fund_value * s_pct, axis=1)
        bond_array = np.insert(bond_return, 0, fund_value * b_pct, axis=1)

        # the last return value is not used so we add a 1 to the end to return the array to its original length
        ones_to_append = np.ones((stock_return.shape[0], 1), dtype=int)
        stock_return = np.append(stock_return, ones_to_append, axis=1)
        bond_return = np.append(bond_return, ones_to_append, axis=1)

        # create pandas series for various calcs
        salary_s = salary_series(**config)

        invest_s = invest_series(salary_s=salary_s, **config)
        invest_s = np.tile(invest_s, (n_scen, 1))

        mortality_s = mortality_sim(**config)

        asset_mix_sb = asset_mix(**config)
   
        asset_mix_s = np.tile(asset_mix_sb['stock'], (n_scen, 1))
        asset_mix_b = np.tile(asset_mix_sb['bond'], (n_scen, 1))
     
        # # Debugging prints
        # print("Type of invest_s:", type(invest_s))
        # # print("First few elements of invest_s:\n", invest_s.head())

        # # Debugging prints
        # print("Type of invest_s:", type(invest_s))
        # print("Shape of invest_s:", invest_s.shape if isinstance(invest_s, (pd.Series, np.ndarray)) else "Not an array or Series")
        # # print("First few elements of invest_s:\n", invest_s.head() if isinstance(invest_s, pd.Series) else invest_s[:5])

        #         # Debugging prints
        # print("Type of asset_mix_s:", type(asset_mix_s))
        # print("Shape of asset_mix_s:", asset_mix_s.shape if isinstance(asset_mix_s, (pd.Series, np.ndarray)) else "Not an array or Series")
        # # print("First few elements of asset_mix_s:\n", asset_mix_s.head() if isinstance(asset_mix_s, pd.Series) else asset_mix_s[:5])

        #         # Debugging prints
        # print("Type of stock_array:", type(stock_array))
        # print("Shape of stock_array:", stock_array.shape if isinstance(stock_array, (pd.Series, np.ndarray)) else "Not an array or Series")
        # # print("First few elements of stock_array:\n", stock_array.head() if isinstance(stock_array, pd.Series) else stock_array[:5])

        # this is where the magic happens
        # calc the fund value at each point in time, credit interest, add/withdraw from fund
        for s, b, inv, alloc_s, alloc_b in zip(stock_array, bond_array, invest_s, asset_mix_s, asset_mix_b):
            for i in range(1, len(s)):      
                s[i] = s[i-1] * s[i]                            # stock fund @t = stock fund @t-1 * stock return
                b[i] = b[i-1] * b[i]                          # bond fund @t = bond fund @t-1 * bond return
                total_fund = s[i] + b[i] + inv[i-1]/freq   # total fund = stock + bond fund +/- investment
                s[i] = total_fund * alloc_s[i]                     # reallocate fund value to stock fund
                b[i] = total_fund * alloc_b[i]                     # reallocate fund value to bond fund

        total_fund = stock_array + bond_array

        df = pd.DataFrame(total_fund.T)

        # Extract the final row
        final_row = df.iloc[-1, :]

        # Rank the columns based on the values in the final row
        ranked_columns = final_row.rank().sort_values()

        # Determine the 5th, 50th, and 95th percentiles
        n = len(ranked_columns)
        p5 = int(0.05 * (n - 1))
        p50 = int(0.50 * (n - 1))
        p95 = int(0.95 * (n - 1))

        percentile_indices = [p5, p50, p95]
        percentile_columns = ranked_columns.iloc[percentile_indices].index

        # Select the percentile columns from the dataframe
        selected_df = df[percentile_columns]

        # Rename columns
        selected_df.columns = ["5th Percentile", "50th Percentile", "95th Percentile"]

        return df

def retirement_analysis(age, ret_age, salary, fund_value, ret_inc):
    analysis = Retirement_Analysis(age, ret_age, salary, fund_value, ret_inc)
    return analysis.fund_projection()

df2 = retirement_analysis(age=30, ret_age=65, salary=100000, fund_value=100000, ret_inc=100000)

output_file_path = '/Users/taylorcastillo/Documents/taylors_castle/RetirementStochastic/Outputs/sample.csv'
df2.to_csv(output_file_path, index=False)
print(df2)

Type of invest_s: <class 'numpy.ndarray'>
Type of invest_s: <class 'numpy.ndarray'>
Shape of invest_s: (100, 66)
Type of asset_mix_s: <class 'numpy.ndarray'>
Shape of asset_mix_s: (100, 66)
Type of stock_array: <class 'numpy.ndarray'>
Shape of stock_array: (100, 66)
Type of invest_s: <class 'numpy.ndarray'>
Type of invest_s: <class 'numpy.ndarray'>
Shape of invest_s: (100, 66)
Type of asset_mix_s: <class 'numpy.ndarray'>
Shape of asset_mix_s: (100, 66)
Type of stock_array: <class 'numpy.ndarray'>
Shape of stock_array: (100, 66)
              0             1             2             3             4   \
0   1.000000e+05  1.000000e+05  1.000000e+05  1.000000e+05  1.000000e+05   
1   1.195736e+05  1.176549e+05  1.232086e+05  1.465420e+05  1.239575e+05   
2   1.689290e+05  1.501909e+05  1.656362e+05  1.612869e+05  1.656745e+05   
3   1.800938e+05  2.105000e+05  1.768777e+05  1.846173e+05  2.631895e+05   
4   2.215290e+05  2.060382e+05  1.886490e+05  2.824785e+05  2.525597e+05   
..        

/Users/taylorcastillo/Documents/taylors_castle/RetirementStochastic/functions.py:93: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  i = row[d] / 1000 * (mi ** (j - age))
/Users/taylorcastillo/Documents/taylors_castle/RetirementStochastic/functions.py:98: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  i = row[d] / 1000 * (mi ** (j - age))
/Users/taylorcastillo/Documents/taylors_castle/RetirementStochastic/functions.py:93: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by